In [ ]:
import  requests as re
from bs4 import BeautifulSoup
from pandas import DataFrame, Series
import numpy as np
import pandas as pd

def new_table(Table):    #기존의 데이터로 예측한 미래 5년의 데이터
    new_t = Table.T.iloc[[16,17,18,19,21]]    #ROA, ROE, EPS, BPS, PER만 추출
    ROA_avg = new_t.mean(axis=1)[0]
    ROE_avg = new_t.mean(axis=1)[1]
    bf_num = -2
    new_column = []
    for i in range(0,6):
        new_year = 2023+i
        new_t['%d'%new_year] = None
        for j in range(2,6):
            if(new_t.loc['BPS'][-j] != 0):
                break
            else:
                bf_num = bf_num-1
        new_EPS = new_t.loc['BPS'][bf_num]*(ROE_avg/100)
        new_BPS = new_t.loc['BPS'][bf_num] + new_EPS
        bf_num = -2
            
        new_t['%d'%new_year] = ROA_avg,ROE_avg,new_EPS, new_BPS, 'NAN'
    return new_t

def calculate_price(new_t):    #마지막 예측년도(2028년)의 BPS를 현재가치로 변환
    new_BPS = new_t.loc['BPS'][-1]
    new_price = new_BPS/((1.15)**8)
    print("업종 PER : ", around_PER)  
    print("현재 주가 :" + price)
    print("현재가치 : " ,round(new_price,2))
    display(new_t)
    print("투자 자격 : ", end="")
    print(("미달", "적합")[1.5*n_price<new_price])   
    
while(True):
    code = input("주식 종목 코드: ")
    if code=="":
        break
    url = re.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%code)
    url = url.content
    html = BeautifulSoup(url,'html.parser')
    body = html.find('body')
    fn_body = body.find('div',{'class':'fng_body asp_body'})
    ur_table = fn_body.find('div',{'id': 'div15'})
    table = ur_table.find('div',{'id':'highlight_D_Y'})
    tbody = table.find('tbody')
    tr = tbody.find_all('tr')
    Table = DataFrame()            #테이블 구성

    price = body.find('span',{'id':'svdMainChartTxt11'}).text #현재 주가 크롤링
    price = price.replace(',','')
    n_price = float(price)
    around_PER = body.find('div',{'id': 'corp_group2'}).find_all('dd')[5].text  #PER 크롤링

    for i in tr:        #테이블 항목 가져오기
        category = i.find('span',{'class':'txt_acd'})
        if(category==None):
            category=i.find('th')
        category = category.text.strip()
        value_list = []
        j = i.find_all('td',{'class','r'})
        
        for value in j:
            temp = value.text.replace(',','').strip()
            try:
                temp = float(temp)
                value_list.append(temp)
            except:
                value_list.append(0)
        Table['%s'%(category)] = value_list

            
        thead = table.find('thead')               #기간 가져오기
        tr_2 = thead.find('tr',{'class','td_gapcolor2'}).find_all('th')
        year_list = []
        for i in tr_2:
            try:
                temp_year = i.find('span', {'class','txt_acd'}).text
            except:
                temp_year = i.text
                
            year_list.append(temp_year)
        Table.index = year_list

    newt = new_table(Table)
    calculate_price(newt)

주식 종목 코드: 035720
업종 PER :  49.16
현재 주가 :324500
현재가치 :  30192.99


,2015/12,2016/12,2017/12,2018/12,2019/12,2020/12(E),2021/12(E),2022/12(E),2023,2024,2025,2026,2027,2028
ROA,2.64,1.51,2.11,0.22,-4.10,3.77,5.00,5.93,2.135,2.135,2.135,2.135,2.135,2.135
ROE,3.02,1.93,2.91,1.04,-5.81,6.33,8.52,10.10,3.505,3.505,3.505,3.505,3.505,3.505
EPS,1269.00,874.00,1601.00,613.00,-3578.00,3888.00,5607.00,7244.00,2632.75,2725.02,2820.54,2919.4,3021.72,3127.63
BPS,42485.00,50777.00,59344.00,61614.00,60615.00,63264.00,68409.00,75114.00,77746.7,80471.8,83292.3,86211.7,89233.4,92361.1
PER,91.59,88.46,85.55,167.97,0.00,83.47,57.88,44.80,NAN,NAN,NAN,NAN,NAN,NAN


C:\Anaconda\envs\py35\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


투자 자격 : 미달


In [9]:
crawling('005930')

21.23


,매출액,영업이익,영업이익(발표기준),당기순이익,지배주주순이익,비지배주주순이익,자산총계,부채총계,자본총계,지배주주지분,...,지배주주순이익률,ROA,ROE,EPS,BPS,DPS,PER,PBR,발행주식수,배당수익률
2015/12,2006535.0,264134.0,264134.0,190601.0,186946.0,3655.0,2421795.0,631197.0,1790598.0,1728768.0,...,9.32,8.07,11.16,2198.0,21903.0,420.0,11.47,1.15,7364967.0,1.67
2016/12,2018667.0,292407.0,292407.0,227261.0,224157.0,3104.0,2621743.0,692113.0,1929630.0,1864243.0,...,11.10,9.01,12.48,2735.0,24340.0,570.0,13.18,1.48,7033967.0,1.58
2017/12,2395754.0,536450.0,536450.0,421867.0,413446.0,8422.0,3017521.0,872607.0,2144914.0,2072134.0,...,17.26,14.96,21.01,5421.0,28971.0,850.0,9.40,1.76,6454925.0,1.67
2018/12,2437714.0,588867.0,588867.0,443449.0,438909.0,4540.0,3393572.0,916041.0,2477532.0,2400690.0,...,18.00,13.83,19.63,6024.0,35342.0,1416.0,6.42,1.10,5969783.0,3.66
2019/12,2304009.0,277685.0,277685.0,217389.0,215051.0,2338.0,3525645.0,896841.0,2628804.0,2549155.0,...,9.33,6.28,8.69,3166.0,37528.0,1416.0,17.63,1.49,5969783.0,2.54
2020/12(E),2319978.0,326077.0,0.0,253590.0,250862.0,0.0,3716756.0,918610.0,2798146.0,2715352.0,...,10.81,7.00,9.53,3693.0,39975.0,1536.0,14.73,1.36,0.0,0.00
2021/12(E),2589048.0,443184.0,0.0,342678.0,338216.0,0.0,4007938.0,975356.0,3032583.0,2945733.0,...,13.06,8.87,11.95,4979.0,43366.0,1576.0,10.93,1.25,0.0,0.00
2022/12(E),2791181.0,489849.0,0.0,378157.0,374562.0,0.0,4347126.0,1051327.0,3295799.0,3204950.0,...,13.42,9.05,12.18,5514.0,47182.0,1565.0,9.87,1.15,0.0,0.00


In [43]:
url = re.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%code)
url = url.content
html = BeautifulSoup(url,'html.parser')
body = html.find('body')
around_PER = body.find('div',{'id': 'corp_group2'}).find_all('dd')[5].text #PER 크롤링

price = body.find('span',{'id': 'svdMainChartTxt11'}).text
price
    


'73,200'